In [24]:
from nltk import edit_distance
from nltk import word_tokenize
from nltk.corpus import stopwords, words
import nltk
import pandas as pd
from  spellchecker import SpellChecker
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from sys import platform

In [25]:
win_path = r"C:\Users\Anton\Desktop\projects\spell_checker\csv_imports\\"
mac_linux_path = "csv_imports/"

In [26]:
if platform == "linux" or platform == "linux2":
    path = mac_linux_path
elif platform == "darwin":
    path = mac_linux_path
elif platform == "win32":
    path = win_path
else:
    path = win_path

In [27]:
porn_terms_df = pd.read_csv(path + "porn_words.csv")

In [28]:
queries_df = pd.read_csv(path + "nlp_queries.csv")
queries_df = queries_df.drop(columns=["#"])

In [29]:
queries_df["query"].unique()
queries_df["query"].nunique()

array(['japanese', 'sexmex', 'hentai', ..., 'colegiala quito',
       'darla caren', 'mistress orgasm'], dtype=object)

1062095

In [30]:
queries_df

,id,query,cnt
0,134,japanese,60332
1,116,sexmex,55632
2,88,hentai,47325
3,360,omegle,46862
4,299,,40304
...,...,...,...
1062092,186075,scoute69stiefsohnspritztab,1
1062093,186081,dean,1
1062094,186193,colegiala quito,1
1062095,186250,darla caren,1


#### Todo
* consider plurals in some cases the same
* make the dictionary more pornographic
* remove correct false-context words
* detect wrong spelled words
* detect keyboard typos

Remove stopwords from queries

In [31]:
stop_words = set(stopwords.words('english')) 

In [32]:
def remove_stop_words(sentence): 
    words = sentence.split() 
    filtered_words = [word for word in words if word not in stop_words] 

    return ' '.join(filtered_words)

In [33]:
spell = SpellChecker()

In [34]:
def spellcheck(query):
    query = remove_stop_words(query)
    query_list = word_tokenize(query)
    mispelled = spell.unknown(query_list)

    result_string = ""
    for word in mispelled:
        porn_words = porn_terms_df["term"].unique()
        if len(list(set(spell.candidates(word)).intersection(set(porn_words)))) == 0:
            result_string += spell.correction(word) + " "
        else:
            # print(spell.candidates(word))
            result_string += list(set(spell.candidates(word)).intersection(set(porn_words)))[0] + " "

    return result_string

In [38]:
spellcheck("bukak")

'bukkake '

In [14]:
query_1 = "japanese milf with big tit"
query_2 = "japanese milf big tits"

In [15]:
spellcheck("japanese mather son")

''

#### Change word frequency

In [16]:
spell.word_frequency.load_text_file(path + "my_uncles_new_wive_part1.txt")

In [17]:
spellcheck("pusy")

{'puss', 'push', 'busy', 'psy', 'jusy', 'pus', 'pussy', 'puny', 'pushy', 'posy'}


'busy '

In [18]:
spell.word_frequency.load_text_file(path + "pussy.txt")

In [19]:
spellcheck("pusy")

{'puss', 'push', 'busy', 'psy', 'jusy', 'pus', 'pussy', 'puny', 'pushy', 'posy'}


'busy '